In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from sklearn.datasets import load_boston
boston_dataset = load_boston()

df = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
df['MEDV'] = boston_dataset.target
df.head()


CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD    TAX  \
0  0.00632  18.0   2.31   0.0  0.538  6.575  65.2  4.0900  1.0  296.0   
1  0.02731   0.0   7.07   0.0  0.469  6.421  78.9  4.9671  2.0  242.0   
2  0.02729   0.0   7.07   0.0  0.469  7.185  61.1  4.9671  2.0  242.0   
3  0.03237   0.0   2.18   0.0  0.458  6.998  45.8  6.0622  3.0  222.0   
4  0.06905   0.0   2.18   0.0  0.458  7.147  54.2  6.0622  3.0  222.0   

   PTRATIO       B  LSTAT  MEDV  
0     15.3  396.90   4.98  24.0  
1     17.8  396.90   9.14  21.6  
2     17.8  392.83   4.03  34.7  
3     18.7  394.63   2.94  33.4  
4     18.7  396.90   5.33  36.2

In [3]:
from sklearn.model_selection import train_test_split

X = boston_dataset.data
y = boston_dataset.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

linear_regression_model = LinearRegression()
linear_regression_model.fit(X_train, y_train)

y_train_pred = linear_regression_model.predict(X_train)
y_test_pred = linear_regression_model.predict(X_test)

mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

print(mse_train)
print(mse_test)

21.641412753226312
24.291119474973456


In [5]:
from sklearn import tree

tree_model = tree.DecisionTreeRegressor(max_depth=3, random_state=42) # CART implementation in sklearn
tree_model = tree_model.fit(X_train, y_train)

y_train_pred = tree_model.predict(X_train)
y_test_pred = tree_model.predict(X_test)

mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

print(mse_train)
print(mse_test)

7.078696760120875
8.553906584646844


In [6]:
from sklearn.tree import export_text

tree_text = export_text(tree_model)
print(tree_text)

|--- feature_5 <= 6.94
|   |--- feature_12 <= 14.40
|   |   |--- feature_7 <= 1.38
|   |   |   |--- feature_0 <= 10.11
|   |   |   |   |--- value: [50.00]
|   |   |   |--- feature_0 >  10.11
|   |   |   |   |--- value: [27.90]
|   |   |--- feature_7 >  1.38
|   |   |   |--- feature_5 <= 6.54
|   |   |   |   |--- feature_5 <= 6.10
|   |   |   |   |   |--- value: [20.36]
|   |   |   |   |--- feature_5 >  6.10
|   |   |   |   |   |--- value: [22.73]
|   |   |   |--- feature_5 >  6.54
|   |   |   |   |--- feature_12 <= 5.78
|   |   |   |   |   |--- value: [30.05]
|   |   |   |   |--- feature_12 >  5.78
|   |   |   |   |   |--- value: [25.99]
|   |--- feature_12 >  14.40
|   |   |--- feature_0 <= 6.93
|   |   |   |--- feature_6 <= 73.30
|   |   |   |   |--- feature_11 <= 395.10
|   |   |   |   |   |--- value: [22.30]
|   |   |   |   |--- feature_11 >  395.10
|   |   |   |   |   |--- value: [19.60]
|   |   |   |--- feature_6 >  73.30
|   |   |   |   |--- feature_0 <= 0.17
|   |   |   |   |  

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn import tree

param_grid = {
    'max_depth': range(1,10)
}

gscv = GridSearchCV(
    tree.DecisionTreeRegressor(random_state=42),
    param_grid,
    cv=10,
    n_jobs=-1
)

gscv.fit(X_train, y_train)

print(gscv.best_params_)

{'max_depth': 5}


In [8]:
from sklearn import tree

tree_model = tree.DecisionTreeRegressor(max_depth=gscv.best_params_['max_depth'], random_state=42)
tree_model = tree_model.fit(X_train, y_train)

y_train_pred = tree_model.predict(X_train)
y_test_pred = tree_model.predict(X_test)

mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

print(mse_train)
print(mse_test)

from sklearn.tree import export_text

tree_text = export_text(tree_model)
print(tree_text)


7.078696760120875
8.553906584646844
|--- feature_5 <= 6.94
|   |--- feature_12 <= 14.40
|   |   |--- feature_7 <= 1.38
|   |   |   |--- feature_0 <= 10.11
|   |   |   |   |--- value: [50.00]
|   |   |   |--- feature_0 >  10.11
|   |   |   |   |--- value: [27.90]
|   |   |--- feature_7 >  1.38
|   |   |   |--- feature_5 <= 6.54
|   |   |   |   |--- feature_5 <= 6.10
|   |   |   |   |   |--- value: [20.36]
|   |   |   |   |--- feature_5 >  6.10
|   |   |   |   |   |--- value: [22.73]
|   |   |   |--- feature_5 >  6.54
|   |   |   |   |--- feature_12 <= 5.78
|   |   |   |   |   |--- value: [30.05]
|   |   |   |   |--- feature_12 >  5.78
|   |   |   |   |   |--- value: [25.99]
|   |--- feature_12 >  14.40
|   |   |--- feature_0 <= 6.93
|   |   |   |--- feature_6 <= 73.30
|   |   |   |   |--- feature_11 <= 395.10
|   |   |   |   |   |--- value: [22.30]
|   |   |   |   |--- feature_11 >  395.10
|   |   |   |   |   |--- value: [19.60]
|   |   |   |--- feature_6 >  73.30
|   |   |   |   |--- f